<a href="https://colab.research.google.com/github/AI-Front/ChatBots/blob/master/Chatbot_on_RuGPT3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPT3 Small общается по-русски
Будем обучать ruGPT-3 small на открытых данных Диалогов [Яндекс.Толоки](https://research.yandex.com/datasets/toloka)


## Установим библиотеки

In [1]:
!pip install torch==1.4.0
!pip3 install transformers==3.5.0

     |████████████████████████████████| 753.4MB 23kB/s 
ERROR: torchvision 0.9.1+cu101 has requirement torch==1.8.1, but you'll have torch 1.4.0 which is incompatible.
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.4.0 which is incompatible.
  Found existing installation: torch 1.8.1+cu101
    Uninstalling torch-1.8.1+cu101:
      Successfully uninstalled torch-1.8.1+cu101
     |████████████████████████████████| 1.3MB 8.5MB/s 
     |████████████████████████████████| 2.9MB 43.7MB/s 
     |████████████████████████████████| 901kB 54.3MB/s 
     |████████████████████████████████| 1.1MB 50.7MB/s 


In [2]:
!git clone  https://github.com/sberbank-ai/ru-gpts

Cloning into 'ru-gpts'...
remote: Enumerating objects: 613, done.
remote: Counting objects: 100% (108/108), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 613 (delta 66), reused 95 (delta 59), pack-reused 505
Receiving objects: 100% (613/613), 360.92 KiB | 13.88 MiB/s, done.
Resolving deltas: 100% (366/366), done.


In [3]:
!mkdir models/

## Скачиваем данные

In [4]:
!wget https://raw.githubusercontent.com/AI-Front/ChatBots/master/dialogues_clean_small.csv

--2021-06-03 07:36:59--  https://raw.githubusercontent.com/AI-Front/ChatBots/master/dialogues_clean_small.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9529360 (9.1M) [text/plain]
Saving to: ‘dialogues_clean_small.csv’

dialogues_clean_sma 100%[===================>]   9.09M  36.5MB/s    in 0.2s    

2021-06-03 07:37:00 (36.5 MB/s) - ‘dialogues_clean_small.csv’ saved [9529360/9529360]



In [5]:
import pandas as pd
data = open('dialogues_clean_small.csv', 'r').readlines()

In [6]:
data[:3]

['dialogue\n',
 '<span class=participant_2>Пользователь 2: Привет) расскажи о себе</span><br /><span class=participant_1>Пользователь 1: Привет) под вкусный кофеек настроение поболтать появилось<br />)</span><br /><span class=participant_2>Пользователь 2: Что читаешь? Мне нравится классика</span><br /><span class=participant_2>Пользователь 2: Я тоже люблю пообщаться</span><br /><span class=participant_1>Пользователь 1: Люблю животных, просто обожаю, как и свою работу)</span><br /><span class=participant_1>Пользователь 1: Я фантастику люблю</span><br /><span class=participant_2>Пользователь 2: А я выращиваю фиалки</span><br /><span class=participant_2>Пользователь 2: И веду здоровый и активный образ жизни!</span><br /><span class=participant_1>Пользователь 1: Ух ты, интересно.</span><br /><span class=participant_2>Пользователь 2: Ты случайно не принц на белом коне? Я его очень жду<br />..</span><br /><span class=participant_1>Пользователь 1: А у меня из хобби каждую неделю тусить с моим

# Почистим наши данные

In [7]:
import os, re
def clean_dialog(text):
  text = re.sub('</span><br />', '', text)
  text = re.sub('<br />', ' ', text)
  text = re.sub('<span class=participant_2>', '\n', text)
  text = re.sub('<span class=participant_1>', '\n', text)
  text = text.strip('\n')
  return text

print(clean_dialog(data[1]))

Пользователь 2: Привет) расскажи о себе
Пользователь 1: Привет) под вкусный кофеек настроение поболтать появилось )
Пользователь 2: Что читаешь? Мне нравится классика
Пользователь 2: Я тоже люблю пообщаться
Пользователь 1: Люблю животных, просто обожаю, как и свою работу)
Пользователь 1: Я фантастику люблю
Пользователь 2: А я выращиваю фиалки
Пользователь 2: И веду здоровый и активный образ жизни!
Пользователь 1: Ух ты, интересно.
Пользователь 2: Ты случайно не принц на белом коне? Я его очень жду ..
Пользователь 1: А у меня из хобби каждую неделю тусить с моим лучшим другом)


In [8]:
dialogs = [clean_dialog(d) for d in data[1:]]

In [9]:
len(dialogs)

3497

In [10]:
train = open('train.txt', 'w')
valid = open('valid.txt', 'w')

for d in dialogs[:500]:
  train.write('<s>'+d+'</s>\n')
train.close()
for d in dialogs[500:1000]:
  valid.write('<s>'+d+'</s>\n')
valid.close()

## Обучаем!
The following code download our model and tokenizer from huggingface and finetune model for generating essays.

This took aroung ten minutes and obtain perplexity = 13-16

In [11]:
!export PYTHONPATH=${PYTHONPATH}:/ru-gpts/
!CUDA_VISIBLE_DEVICES=0 python ru-gpts/pretrain_transformers.py \
    --output_dir=models/chatbot \
    --model_type=gpt2 \
    --model_name_or_path=sberbank-ai/rugpt3small_based_on_gpt2 \
    --do_train \
    --train_data_file=train.txt \
    --do_eval \
    --eval_data_file=valid.txt \
    --per_gpu_train_batch_size 1 \
    --gradient_accumulation_steps 1 \
    --num_train_epochs 5 \
    --block_size 2048 \
    --overwrite_output_dir

2021-06-03 07:37:03.221059: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
06/03/2021 07:37:05 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
06/03/2021 07:37:06 - INFO - filelock -   Lock 140128742805328 acquired on /root/.cache/torch/transformers/06f48b6b3173390d047e15d691fda67ae4ea7733a5eea4b6e0115f5099c4e700.b5cdfa39c63384f94159c36bc9042660c747cea5cf520b43d543bd2c68b3164d.lock
Downloading: 100% 608/608 [00:00<00:00, 309kB/s]
06/03/2021 07:37:06 - INFO - filelock -   Lock 140128742805328 released on /root/.cache/torch/transformers/06f48b6b3173390d047e15d691fda67ae4ea7733a5eea4b6e0115f5099c4e700.b5cdfa39c63384f94159c36bc9042660c747cea5cf520b43d543bd2c68b3164d.lock
06/03/2021 07:37:07 - INFO - filelock -   Lock 140128890132624 acquired on /root/.cache/torch/transformers/1b36eeb1fd7b3a6ec11bf46bde2c38e7e68f71ec774694b9e886c86001aab35d.c483bc344

## Оценим результаты


### Генерируем вручную

In [28]:
import numpy as np
import torch
np.random.seed(42)
torch.manual_seed(42)
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from nltk import sent_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [14]:
tok = GPT2Tokenizer.from_pretrained("models/chatbot")
model = GPT2LMHeadModel.from_pretrained("models/chatbot")
model.cuda()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Laye

In [32]:
def clean_text(messages=[]):
  for i in range(len(messages)):
    if i% 2 == 0:
      messages[i] = 'Пользователь 1:'+messages[i]
      last = 'Пользователь 1:'
    else:
      messages[i] = 'Пользователь 2:'+messages[i]
      last = 'Пользователь 2:'
  messages = "<s>"+'\n'.join(messages)
  if last=='Пользователь 2:':
    last = 'Пользователь 1:'
    messages+='\nПользователь 1:'
  else:
    last = 'Пользователь 2:'
    messages+='\nПользователь 2:'
  return messages, last


In [33]:
def generate_dialog(messages):
  message, last = clean_text(messages)
  inpt = tok.encode(message, return_tensors="pt")
  out = model.generate(inpt.cuda(), max_length=500, repetition_penalty=5.0, do_sample=True, top_k=5, top_p=0.95, temperature=1)
  text = tok.decode(out[0])
  #print(last)
  text = re.split(last, text)[-1]
  text = ' '.join(sent_tokenize(text)[:6]
                  )
  return text.split('\n')[0]

In [34]:
messages = ["Привет! Как дела?"]
print(generate_dialog(messages))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 Привет, отлично. Чем занимаешься?) Я работаю учителем математики) а ты чем любишь заниматься?)) А я люблю готовить. И у меня есть котёнок ^_~😊👍❤️ ✋""" 
 5 лет назад мы переехали в город Сочи, там и познакомились с новыми друзьями )) Сейчас живем на улице Мира (сейчас проспект Ленина д 161 ) очень уютная атмосфера ))) Очень нравится эта улица для прогулок по вечерам :) Люблю проводить время со своим любимым котиком Мишей))) Он такой милый... пушистый :3 </smfi] СПА-ЛЯРТЕ! МНЕ ПОДРАЖИЛОСЬ У ТЕБЯ С КОРОЛЕМ МИШЕЙ!!!


In [35]:
messages = ["Привет! Как дела?",
            "Привет! Все хорошо, а у тебя?"]
print(generate_dialog(messages))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Хорошо. Ты откуда и где?) Чем занимаешься по жизни?) А я из Москвы. Учусь в школе на инженера 😂 Вот скоро иду домой к родителям, они ждут меня ) Люблю готовить вкусно )) Вкусно поесть это здорово! Особенно когда есть семья с маленькими детьми ))) Я обожаю сладкое... Да ещё люблю мандарины ❤️☺👍‼А ты чем увлекаешся помимо учебы/работы)?
